# A Brief Introduction to AuthoritySpoke

> "Details, unnumbered, shifting, sharp, disordered, unchartable, jagged."
>
> Karl N. Llewellyn, The Bramble Bush: On Our Law and Its Study (1930).
    
AuthoritySpoke is a Python package that will help you make legal authority readable by computers.

This notebook will provide an overview of AuthoritySpoke's most important features. AuthoritySpoke is still in an early alpha state, so many features have yet to be implemented, and some others still have limited functionality.

## 1. Importing the Package

If you want to use AuthoritySpoke in your own Python environment, be sure you have installed AuthoritySpoke using a command like `pip install AuthoritySpoke` on the command line. Visit [the Python Package Index](https://pypi.org/project/AuthoritySpoke/) for more details.

With a Python environment activated, let's import AuthoritySpoke.

In [1]:
import authorityspoke

If you executed that cell with no error messages, then it worked! 

If you got a message like `No module named 'authorityspoke'`, then AuthoritySpoke is probably not installed in your current Python environment. In that case, check the [Python documentation](https://docs.python.org/3/installing/index.html) for help.

## 2. Downloading and Importing Decisions

Now we need some court opinions to load into AuthoritySpoke. We can collect these from the Case Access Project API, a project of the Harvard Law School Library Innovation Lab. To download full cases from CAP, you'll need to [register for an API key](https://case.law/user/register/). However, if you'd rather skip forward to the end of this section without interacting with the API, then just go to section 2.1 below. There's already a copy of the files we're going to download in the "example_data/opinions" folder of this repository.

The CAP API limits users to downloading 500 full cases per day. If you accidentally make a query that returns hundreds of full cases, you could hit your limit before you know it. You should first try out your API queries without the "full_case": "true" parameter, and then only request full cases once you're confident you'll receive what you expect. To minimize the risk of revealing your API key to others, you can store it in an environment variable called CAP_API_KEY and then retrieve it as shown in the cell below.

If you're viewing this tutorial in a cloud environment like Binder, you could either replace `os.environ['CAP_API_KEY']` with a string containing your own API key, or skip the use of the API key as described below in section 2.1.

In [2]:
import os

CAP_API_KEY = os.environ['CAP_API_KEY']

Next we need to download some cases for analysis.

Let's download _Oracle America v. Google_, 750 F.3d 1339 (2014), a landmark opinion in which the Federal Circuit Court of Appeals held that the API declarations for the Java language were copyrightable. And since we'll want to compare the _Oracle_ case to a related case, let's also download _Lotus Development Corporation v. Borland International_, 49 F.3d 807 (1995). In that case, the First Circuit Court of Appeals held that the menu structure of a spreadsheet program called Lotus 1-2-3 was uncopyrightable because it was a "method of operation" under the Copyright Act. As we'll see, the _Oracle_ case discusses and disagrees with the _Lotus_ case.


In [3]:
from authorityspoke.io.downloads import download_case

oracle_download = download_case(cite="750 F.3d 1339", filename="oracle_h.json")

Now we have a record representing the _Oracle_ case, which has been saved to the "example_data/opinions" folder under the filename "oracle_h.json". Let's look at the API response.

In [4]:
from pprint import pprint
pprint(oracle_download)

{'citations': [{'cite': '750 F.3d 1339', 'type': 'official'}],
 'court': {'id': 8955,
           'name': 'United States Court of Appeals for the Federal Circuit',
           'name_abbreviation': 'Fed. Cir.',
           'slug': 'fed-cir',
           'url': 'https://api.case.law/v1/courts/fed-cir/'},
 'decision_date': '2014-05-09',
 'docket_number': 'Nos. 2013-1021, 2013-1022',
 'first_page': '1339',
 'frontend_url': 'https://cite.case.law/f3d/750/1339/',
 'id': 4066790,
 'jurisdiction': {'id': 39,
                  'name': 'U.S.',
                  'name_long': 'United States',
                  'slug': 'us',
                  'url': 'https://api.case.law/v1/jurisdictions/us/',
                  'whitelisted': False},
 'last_page': '1381',
 'name': 'ORACLE AMERICA, INC., Plaintiff-Appellant, v. GOOGLE INC., '
         'Defendant-Cross-Appellant',
 'name_abbreviation': 'Oracle America, Inc. v. Google Inc.',
 'reporter': {'full_name': 'Federal Reporter 3d Series',
              'id': 933,

Yes, this is the case I expected. But if I had provided my API key and used the full_case flag, I could have received more information, like whether there are any non-majority opinions in the case, and the names of the opinion authors. So let's request the _Oracle_ case with `full_case=True`.

In [5]:
oracle_download = download_case(
    cite="750 F.3d 1339", 
    filename="oracle_h.json", 
    full_case=True, 
    api_key=CAP_API_KEY)

And then do the same for the _Lotus_ case.

In [6]:
lotus_download = download_case(
    cite="49 F.3d 807", 
    filename="lotus_h.json", 
    full_case=True, 
    api_key=CAP_API_KEY)

Now let's convert the _Oracle_ API response to an AuthoritySpoke object.

In [7]:
from authorityspoke.io.readers import read_decision

oracle = read_decision(oracle_download)

And take a look at the object we made.

In [8]:
print(oracle)

Oracle America, Inc. v. Google Inc., 750 F.3d 1339 (2014-05-09)


In [9]:
lotus = read_decision(lotus_download)
print(lotus)

Lotus Development Corp. v. Borland International, Inc., 49 F.3d 807 (1995-03-09)


Finally, what should you do if you chose not to get an API key or were unable to create the Decision objects from downloaded data? Use the following commands to create the Decision objects from the files in the `example_data/cases` folder.

If you already did the steps above, you can skip the next cell and go to section 3.

## 2.1 Skip the Download and Load Decisions from File

In [10]:
# If you already downloaded Opinions from the API,
# running this cell will overwrite them with example data.
# You should be able to use the rest of the notebook either way.

from authorityspoke.io.loaders import load_and_read_decision

oracle = load_and_read_decision("oracle_h.json")
lotus = load_and_read_decision("lotus_h.json")

## 3. Importing Codes

AuthoritySpoke does not currently interface with any API to retrieve legislative codes, the way it connects to the CAP API to retrieve case opinions. However, AuthoritySpoke can import legislative XML files as Code objects ("Code" in the sense of a legislative code), if the XML adheres to the United States Legislative Markup (USLM) format as used by the United States Code. Although AuthoritySpoke does have functions to import federal regulations and California statutes, which are not published in USLM, those functions are still brittle and are currently only suitable for creating test data.


In [11]:
from authorityspoke.io.loaders import load_and_read_code

constitution = load_and_read_code("constitution.xml")
usc17 = load_and_read_code("usc17.xml")
cfr37 = load_and_read_code("cfr37.xml")

When multiple Codes are enacted in one country's legal system, the best way to organize the Code objects is to create a Regime object representing the country and link each of the Codes to the Regime object.

In [12]:
from authorityspoke import Regime

usa = Regime()

usa.set_code(constitution)
usa.set_code(usc17)
usa.set_code(cfr37)


One judicial `Decision` can include multiple `Opinion`s. The _Lotus_ Decision has a concurring opinion as well as a majority opinion. Access the `majority` attribute of the Decision object to get the majority opinion.

In [13]:
print(lotus.majority)

majority Opinion by STAHL, Circuit Judge.


## 4. Importing and Exporting Legal Holdings

Now we can link some legal analysis to each majority `Opinion` by using `Decision.posit` or `Opinion.posit`. The parameter we pass to this function is a `Holding` or list of `Holding`s posited by the `Opinion`. A **Holding** is statement about whether a **Rule** is or is not valid law. A `Holding` may exist in the abstract, or it may be **posited** by one or more `Opinion`s, which means that the `Opinion` adopts the `Holding` as its own. An `Opinion` may posit more than one `Holding`.

Sadly, the labor of creating data about `Holding`s falls mainly to the user rather than the computer, at least in this early version of AuthoritySpoke. AuthoritySpoke loads `Holding`s from structured descriptions that need to be created outside of AuthoritySpoke as JSON files.

An explanation of the interface for creating new `Holding` objects can be found in the `create_holding_data` notebook in this folder.

For now, this introduction will rely on example JSON files that have already been created. AuthoritySpoke should find them and convert them to AuthoritySpoke objects when we call the `load_and_read_holdings` function. If you pass in a `regime` parameter, AuthoritySpoke can use it to find and link the statutes or other `Enactment`s cited in the `Holding`.


In [14]:
from authorityspoke.io.loaders import load_and_read_holdings

oracle_holdings = load_and_read_holdings("holding_oracle.json", regime=usa)
print(oracle_holdings[0])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact it is false that <the Java API> was an original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


You can also convert Holdings back to JSON using the ``dump`` module.

In [15]:
from authorityspoke.io.dump import to_json

to_json(oracle_holdings[0])

'{"decided": true, "rule_valid": true, "generic": false, "rule": {"enactments_despite": [], "universal": false, "enactments": [{"selector": {"prefix": "", "exact": "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device.", "suffix": ""}, "source": "/us/usc/t17/s102/a", "name": "copyright protection provision"}], "name": null, "procedure": {"outputs": [{"absent": false, "context_factors": [{"generic": true, "plural": false, "name": "the Java API", "type": "Entity"}], "predicate": {"quantity": null, "reciprocal": false, "truth": false, "content": "{} was copyrightable", "comparison": ""}, "name": "false the Java API was copyrightable", "generic": false, "standard_of_proof": null, "type": "Fact"}], "inputs": [{"absent": false, "context_factors": [{"gen

## 5. Linking Holdings to Opinions

If you want annotation anchors to link each Holding to a passage in the Opinion, you can use the ``load_holdings_with_anchors`` method. The result is type of NamedTuple called a HoldingIndexed. You can pass this NamedTuple as the only argument to the `Opinion.posit()` method to assign the `Holding`s to the majority `Opinion`. This will also link the correct text passages from the Opinion to each Holding.

In [16]:
from authorityspoke.io.loaders import load_holdings_with_anchors

oracle_holdings_with_anchors = load_holdings_with_anchors("holding_oracle.json", regime=usa)
lotus_holdings_with_anchors = load_holdings_with_anchors("holding_lotus.json", regime=usa)

oracle.posit(oracle_holdings_with_anchors)
lotus.posit(lotus_holdings_with_anchors)

You can pass either one Holding or a list of Holdings to `Opinion.posit()`. The `Opinion.posit()` method also has a `text_links` parameter that takes a dict indicating what text spans in the Opinion should be linked to which Holding.

## 6. Viewing an Opinion's Holdings

If you take a look in "holding_oracle.json", you'll see that it's a list of 20 holdings. (You can verify this by checking how many times the string "inputs" appears in the file.)

Let's make sure that the .posit() method linked all of those holdings to our `oracle` Opinion object.

In [17]:
len(oracle.holdings)

20

Now let's see the string representation of the AuthoritySpoke Holding object we created from the structured JSON we saw above.

In [18]:
print(oracle.holdings[0])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact it is false that <the Java API> was an original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


Instead of the terms "inputs" and "outputs" we saw in the JSON file, we now have "GIVEN" and "RESULT". And the "RESULT" comes first, because it's hard to understand anything else about a legal rule until you understand what it does. Also, notice the separate heading "GIVEN the ENACTMENT". This indicates that the existence of statutory text (or another kind of enactment such as a constitution) can also be a precondition for a `Rule` to apply. So the two preconditions that must be present to apply this `Rule` are "the Fact it is false that the Java API was an original work" and the statutory text creating copyright protection.

It's also important to notice that a `Rule` can be purely hypothetical from the point of view of the Opinion that posits it. In this case, the court finds that there would be a certain legal significance if it was "GIVEN" that `it is false that <the Java API> was an original work`, but the court isn't going to find that precondition applies, so it's also not going to accept the "RESULT" that `it is false that <the Java API> was copyrightable`.

We can also access just the inputs of a `Holding`, just the `Enactment`s, etc.

In [19]:
print(oracle.holdings[0].inputs[0])

the Fact it is false that <the Java API> was an original work


In [20]:
print(oracle.holdings[0].enactments[0])

"Copyright protection subsists, in accordance with this title, in
original works of authorship fixed in any tangible medium of
expression, now known or later developed, from which they can be
perceived, reproduced, or otherwise communicated, either directly or
with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


## 7. Generic Factors

The two instances of the phrase "the Java API" are in angle brackets to indicate that the Java API is a generic `Entity` mentioned in the `Fact`.

In [21]:
oracle.holdings[0].generic_factors

[Entity(name='the Java API', generic=True, plural=False)]

A generic `Entity` is "generic" in the sense that in the context of the `Factor` where the `Entity` appears, it could be replaced with some other generic `Entity` without changing the meaning of the `Factor` or the `Rule` where it appears.

Let's illustrate this idea with the first `Holding` from the _Lotus_ case.

In [22]:
print(lotus.holdings[0])

the Holding to ACCEPT that the EXCLUSIVE way to reach the fact that
<Borland International> infringed the copyright in <the Lotus menu
command hierarchy> is
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Borland International> infringed the copyright in <the
      Lotus menu command hierarchy>
    GIVEN:
      the Fact that <the Lotus menu command hierarchy> was copyrightable
      the Fact that <Borland International> copied constituent elements of
      <the Lotus menu command hierarchy> that were original
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


What if we wanted to generalize this `Holding` about copyright and apply it in a different context, maybe involving books or movies instead of computer programs? First we could look at the "generic" `Factor`s of the `Holding`, which were marked off in angle brackets in the string representation of the `Holding`.

In [23]:
lotus.holdings[0].generic_factors

[Entity(name='Borland International', generic=True, plural=False),
 Entity(name='the Lotus menu command hierarchy', generic=True, plural=False)]

The same `Rule`s and `Holding`s may be relevant to more than one `Opinion`. Let's try applying the idea from `lotus.holdings[0]` to a different copyright case that's also about a derivative work. In [_Castle Rock Entertainment, Inc. v. Carol Publishing Group Inc._](https://en.wikipedia.org/wiki/Castle_Rock_Entertainment,_Inc._v._Carol_Publishing_Group_Inc.) (1998), a United States Court of Appeals found that a publisher infringed the copyright in the sitcom _Seinfeld_ by publishing a trivia book called _SAT: The Seinfeld Aptitude Test_.

Maybe we'd like to see how the `Holding` from the _Lotus_ case could have applied in the context of the _Castle Rock Entertainment_ case, under 17 USC 102. We can check that by using the `Holding.new_context()` method to replace the generic factors from the _Lotus_ `Holding`.

In [24]:
from authorityspoke import Entity

seinfeld_holding = lotus.holdings[0].new_context(
    {Entity('Borland International'): Entity('Carol Publishing Group'),
    Entity('the Lotus menu command hierarchy'): Entity("Seinfeld")}
)

In AuthoritySpoke, Holding and Factor objects are "frozen" objects, which means Python will try to prevent you from modifying the object after it has been created. The `new_context` method returns a new `Holding` object, which we've assigned to the name `seinfeld_holding`, but the `Holding` that we used as a basis for the new object also still exists, and it's unchanged.

In [25]:
print(seinfeld_holding)

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Carol Publishing Group> infringed the copyright in
      <Seinfeld>
    GIVEN:
      the Fact that <Seinfeld> was copyrightable
      the Fact that <Carol Publishing Group> copied constituent elements of
      <Seinfeld> that were original
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


Even though these `Holding`s have different generic factors and don't evaluate equal to one another, the `Holding.means()` method shows that they have the same meaning. In other words, they both endorse exactly the same legal Rule. If Holding A `means` Holding B, then Holding A also necessarily `implies` Holding B.

In [26]:
lotus.holdings[0] == seinfeld_holding

False

In [27]:
lotus.holdings[0].means(seinfeld_holding)

True

## 8. Enactment Objects and Implication

Sometimes it's useful to know whether one `Rule` or `Holding` implies another. Basically, one legal `Holding` implies a second `Holding` if its meaning entirely includes the meaning of the second `Holding`. To illustrate this idea, let's look at the `Enactment` that needs to be present to trigger the `Holding` at `oracle.holdings[0]`.

In [28]:
copyright_provision = oracle.holdings[0].enactments[0]
print(copyright_provision)

"Copyright protection subsists, in accordance with this title, in
original works of authorship fixed in any tangible medium of
expression, now known or later developed, from which they can be
perceived, reproduced, or otherwise communicated, either directly or
with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


The `Enactment` object refers to a `Code` object, which is an instance of an AuthoritySpoke class representing a code of laws. Specifically, it refers to [Title 17 of the United States Code](https://www.copyright.gov/title17/).

In [29]:
usc = copyright_provision.code
print(usc)

USC Title 17


Next, let's create a new `Enactment` object representing a shorter passage of text from the same `Code`.

In [30]:
from authorityspoke import Enactment
from anchorpoint import TextQuoteSelector

works_of_authorship_selector = TextQuoteSelector(
        
        exact=("Copyright protection subsists, in accordance with this title,"
                  + " in original works of authorship")
        )


works_of_authorship_clause = Enactment(
           source="/us/usc/t17/s102/a", selector=works_of_authorship_selector,
            code=usc
)

Now we can create a new `Holding` object that cites to our new `Enactment` object rather than the old one. This time, instead of using the `new_context` method to create a new `Holding` object, we'll use the `evolve` method. With the `evolve` method, instead of specifying `Factor`s that should be replaced wherever they're found, we specify which attributes of the `Rule` object we want to replace, and then specify what we want to replace those attributes' old values with. This returns a new `Holding` object and doesn't change the existing `Holding`.

In [31]:
rule_with_shorter_enactment = oracle.holdings[0].evolve(
            {"enactments": works_of_authorship_clause}
        )

In [32]:
print(rule_with_shorter_enactment)

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact it is false that <the Java API> was an original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship" (Title 17, /us/usc/t17/s102/a)


Now let's try comparing this new `Rule` with the real `Rule` from the _Oracle_ case, to see whether one implies the other. When you're comparing AuthoritySpoke objects, the greater than sign `>` means "implies, but is not equal to".

In [33]:
rule_with_shorter_enactment > oracle.holdings[0]

True

You can also use the greater than or equal sign `>=` to mean "implies or is equal to". In logic, it's common to say that identical statements also imply one another, so that would mean `>=` is the symbol that really means "implies". `<=` can also be used, and it means "is implied by or is equal to".

In [34]:
rule_with_shorter_enactment <= oracle.holdings[0]

False

By comparing the string representations of the original `Rule` from the _Oracle_ case and `rule_with_shorter_enactment`, can you tell why the latter implies the former, and not the other way around?

If you guessed that it was because `rule_with_shorter_enactment` has a shorter `Enactment`, you're right. `Rule`s that require fewer, or less specific, inputs are _broader_ than `Rule`s that have more inputs, because there's a larger set of situations where those `Rule`s can be triggered.

If this relationship isn't clear to you, imagine some "Enactment A" containing only a subset of the text of "Enactment B", and then imagine what would happen if a legislature amended some of the statutory text that was part of Enactment B but not of Enactment A. A requirement to cite Enactment B would no longer be possible to satisfy, because some of that text would no longer be available. Thus a requirement to cite Enactment A could be satisfied in every situation where a requirement to cite Enactment B could be satisfied, and then some.

## 9. Checking for Contradictions

Let's turn back to the _Lotus_ case.

It says that under a statute providing that "In no case does copyright protection for an original work of authorship extend to any...method of operation", the fact that a Lotus menu command hierarchy was a "method of operation" meant that it was also uncopyrightable, despite a couple of `Fact`s that might tempt some courts to rule the other way.

In [35]:
print(lotus.holdings[6])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact it is false that <the Lotus menu command hierarchy> was
      copyrightable
    GIVEN:
      the Fact that <the Lotus menu command hierarchy> was a method of
      operation
    DESPITE:
      the Fact that a text described <the Lotus menu command hierarchy>
      the Fact that <the Lotus menu command hierarchy> was an original work
    GIVEN the ENACTMENTS:
      "In no case does copyright protection for an original work of
      authorship extend to any" (Title 17, /us/usc/t17/s102/b)
      "method of operation" (Title 17, /us/usc/t17/s102/b)


_Lotus_ was a case relied upon by Google in the _Oracle v. Google_ case, but Oracle was the winner in that decision. So we might wonder whether the _Oracle_ Opinion contradicts the _Lotus_ Opinion. Let's check.

In [36]:
oracle.contradicts(lotus)

True

That's good to know, but we don't want to take it on faith that a contradiction exists. Let's use the `explain_contradiction` method to find the contradictory `Holding`s posited by the _Oracle_ and _Lotus_ cases, and to generate a rudimentary explanation of why they contradict.

In [37]:
explanation = lotus.explain_contradiction(oracle)
print(explanation)

an Explanation of why there is a CONTRADICTION between
  the Holding to ACCEPT
    the Rule that the court MUST ALWAYS impose the
      RESULT:
        the Fact it is false that <the Lotus menu command hierarchy> was
        copyrightable
      GIVEN:
        the Fact that <the Lotus menu command hierarchy> was a method of
        operation
      DESPITE:
        the Fact that a text described <the Lotus menu command hierarchy>
        the Fact that <the Lotus menu command hierarchy> was an original work
      GIVEN the ENACTMENTS:
        "In no case does copyright protection for an original work of
        authorship extend to any" (Title 17, /us/usc/t17/s102/b)
        "method of operation" (Title 17, /us/usc/t17/s102/b)
and
  the Holding to ACCEPT
    the Rule that the court MUST SOMETIMES impose the
      RESULT:
        the Fact that <the Java API> was copyrightable
      GIVEN:
        the Fact that <the Java language> was a computer program
        the Fact that <the Java API> 

That's a really complicated holding! Good thing we have AuthoritySpoke to help us grapple with it.

We can use the `explanations_contradiction` method directly on `Holding`s to generate all available "explanations" of why a contradiction is possible between these lotus.holdings[6] and oracle.holdings[10]. Each `Explanation` includes a mapping that shows how the context factors of the `Holding` on the left can be mapped onto the `Holding` on the right. The explanation we've already been given is that these two `Holding`s contradict each other if you consider 'the Lotus menu command hierarchy' to be analagous to 'the Java API'. The other possible explanation AuthoritySpoke could have given would have been that 'the Lotus menu command hierarchy' is analagous to 'the Java language'. Let's see if the other possible `Explanation` also appears in `explanations`. (The `explain_contradiction` method returns only one one `Explanation`\, but `explanations_contradiction` returns all it can find.)

In [38]:
explanations = list(lotus.holdings[6].explanations_contradiction(oracle.holdings[10]))
len(explanations)

1

No, there's only the one explanation of how these rules can contradict each other. If you read the _Oracle_ case, this makes sense. It's only about infringing the copyright in the Java API, not the copyright in the whole Java language. A statement about infringement of 'the Java language' would be irrelevant, not contradictory.

But what exactly is the contradiction between the two `Holding`s?

The first obvious contrast between `lotus.holdings[6]` and `oracle.holdings[10]` is that the `Holding` from the _Lotus_ case is relatively succinct and categorical. The _Lotus_ court interprets Section 102(b) of the Copyright Act to mean that if a work is a "method of operation", it's simply impossible for that work to be copyrighted, so it's not necessary to consider a lot of case-specific facts to reach a conclusion.

The Federal Circuit's _Oracle_ decision complicates that view significantly. The Federal Circuit believes that the fact that an API is, or hypothetically might be, a "method of operation" is only one of many factors that a court can consider in deciding copyrightability. The following quotation, repeated in the _Oracle_ case, illustrates the Federal Circuit's view.

> “Section 102(b) does not extinguish the protection accorded a particular expression of an idea merely because that expression is embodied in a method of operation.” _Mitel, Inc. v. Iqtel, Inc._, 124 F.3d 1366, 1372 (10th Cir.1997)

And that's why AuthoritySpoke finds a contradiction between these two `Rule`s. The _Oracle_ opinion says that courts can sometimes accept the result `the Fact that <the Java API> was copyrightable` despite the `Fact` `<the Java API> was a method of operation`. The _Lotus_ Opinion would consider that impossible.

By the way, AuthoritySpoke isn't applying any sophisticated grammatical parsing to understand the meaning of each Fact. AuthoritySpoke mostly won't recognize that `Fact`s have the same meaning unless their `content` values are exactly the same string. As discussed above, they can also differ in their references to generic factors, which are the phrases that appear in brackets when you use the `print()` command on them. And AuthoritySpoke can also compare `Fact`s based on an optional numeric value that can come at the end of their content, but that feature isn't demonstrated in this tutorial.

## 10. Adding Holdings to One Another

To try out the addition operation, let's load another case from the `example_data` folder.

In [39]:
feist = load_and_read_decision("feist_h.json")
feist.posit(load_holdings_with_anchors("holding_feist.json", regime=usa))


[_Feist Publications, Inc. v. Rural Telephone Service Co._](https://en.wikipedia.org/wiki/Feist_Publications,_Inc.,_v._Rural_Telephone_Service_Co.) was a case that held that the listings in a telephone directory did not qualify as "an original work" and that only original works are eligible for protection under the Copyright Act. This is a two-step analysis. 

The first step results in the Fact it is false that a generic Entity was "an original work": 

In [40]:
print(feist.holdings[10])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact it is false that <Rural's telephone listings> were an
      original work
    GIVEN:
      the Fact that <Rural's telephone listings> were names, towns, and
      telephone numbers of telephone subscribers
    GIVEN the ENACTMENTS:
      "To promote the Progress of Science and useful Arts, by securing for
      limited Times to Authors" (Constitution of the United States,
      /us/const/article-I/8/8)
      "the exclusive Right to their respective Writings" (Constitution of
      the United States, /us/const/article-I/8/8)
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship" (Title 17, /us/usc/t17/s102/a)
      "The copyright in a compilation" (Title 17, /us/usc/t17/s103/b)
      "extends only to the material contributed by the author of such work,
      as distinguished from the preexisting material employed in the work,
      and 

And the second step relies on the result of the first step to reach the further result of "absence of the Fact that" a generic Entity was "copyrightable".

In [41]:
print(feist.holdings[3])

the Holding to ACCEPT that the EXCLUSIVE way to reach the fact that
<Rural's telephone directory> was copyrightable is
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Rural's telephone directory> was copyrightable
    GIVEN:
      the Fact that <Rural's telephone directory> was an original work
    GIVEN the ENACTMENTS:
      "To promote the Progress of Science and useful Arts, by securing for
      limited Times to Authors" (Constitution of the United States,
      /us/const/article-I/8/8)
      "the exclusive Right to their respective Writings" (Constitution of
      the United States, /us/const/article-I/8/8)
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship" (Title 17, /us/usc/t17/s102/a)


In this situation, anytime the first Holding (feist.holdings[10]) is applied, the second Holding (feist.holdings[3]) can be applied as well. That means the two Holdings can be added together to make a single Holding that captures the whole process.

In [42]:
listings_not_copyrightable = feist.holdings[10] + feist.holdings[3]
print(listings_not_copyrightable)

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact it is false that <Rural's telephone listings> were an
      original work
      absence of the Fact that <Rural's telephone listings> were
      copyrightable
    GIVEN:
      the Fact that <Rural's telephone listings> were names, towns, and
      telephone numbers of telephone subscribers
    GIVEN the ENACTMENTS:
      "To promote the Progress of Science and useful Arts, by securing for
      limited Times to Authors" (Constitution of the United States,
      /us/const/article-I/8/8)
      "the exclusive Right to their respective Writings" (Constitution of
      the United States, /us/const/article-I/8/8)
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship" (Title 17, /us/usc/t17/s102/a)
      "The copyright in a compilation" (Title 17, /us/usc/t17/s103/b)
      "extends only to the material contributed by the author of such work,


The difference between `feist.holdings[10]` and the newly-created Holding `listings_not_copyrightable` is that `listings_not_copyrightable` has two Factors under its "RESULT", not just one. Notice that it doesn't matter that the two original Holdings reference different generic Entities ("Rural's telephone directory" versus "Rural's telephone listings"). Because they're generic, they're interchangeable for this purpose.

You might recall that oracle.holdings[0] also was also about the relationship between originality and copyrightability. Let's see what happens when we add oracle.holdings[0] to feist.holdings[10].

In [43]:
print(feist.holdings[10] + oracle.holdings[0])

None


Can you guess why it's not possible to add these two Holdings together? Here's a hint:

In [44]:
feist.holdings[10].exclusive

False

In [45]:
oracle.holdings[0].exclusive

False

In [46]:
feist.holdings[3].exclusive

True

`feist.holdings[10]` and `oracle.holdings[0]` are both Holdings that purport to apply in only "SOME" cases where the specified inputs are present, while `feist.holdings[3]` purports to be the "EXCLUSIVE" way to reach its output, which indicates a statement about "ALL" cases.

You can't infer that there's any situation where `feist.holdings[10]` and `oracle.holdings[0]` can actually be applied together, because there might not be any overlap between the "SOME" cases where one applies and the "SOME" cases where the other applies. But if `feist.holdings[10]` and `feist.holdings[3]` are both valid law, then we know they can both apply together in any of the "SOME" cases where `feist.holdings[10]` applies.

## 11. Set Operations with Holdings

In AuthoritySpoke, the union operation is different from the addition operation, and it usually gives different results.

In [47]:
result_of_adding = feist.holdings[10] + feist.holdings[3]
result_of_union = feist.holdings[10] | feist.holdings[3]

result_of_adding == result_of_union

False

Two set operations that can be meaningfully applied to AuthoritySpoke objects are the union operation (using Python's | operator) and the intersection operation (not yet implemented in AuthoritySpoke 0.3).

For context, let's review how these operators apply to ordinary Python sets. The union operator combines two sets by returning a new set with all of the elements of either of the original sets.

In [48]:
{3, 4} | {1, 4, 5}

{1, 3, 4, 5}

The intersection operator returns a new set with only the elements that were in both original sets.

In [49]:
{3, 4} & {1, 4, 5}

{4}

Apply the union operator to two `Holding`s to get a new `Holding` with all of the inputs and all of the outputs of both of the two original `Holding`s. However, you only get such a new `Holding` if it can be inferred by accepting the truth of the two original `Holding`s. If the two original `Holding`s contradict one another, the operation returns `None`. Likewise, if the two original `Holding`s both have the value `False` for the parameter `universal`, the operation will return `None` if it's possible that the "SOME" cases where one of the original `Holding`s applies don't overlap with the "SOME" cases where the other applies.

In this example, we'll look at a `Holding` from _Oracle_, then a `Holding` from _Feist_, and then the union of both of them.

In [50]:
print(oracle.holdings[1])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was an original work
    GIVEN:
      the Fact that <the Java API> was independently created by the author,
      as opposed to copied from other works
      the Fact that <the Java API> possessed at least some minimal degree of
      creativity
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


In [51]:
print(feist.holdings[2])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact it is false that <Rural's telephone directory> was
      copyrightable
    GIVEN:
      the Fact that <Rural's telephone directory> was an idea
    GIVEN the ENACTMENTS:
      "To promote the Progress of Science and useful Arts, by securing for
      limited Times to Authors" (Constitution of the United States,
      /us/const/article-I/8/8)
      "the exclusive Right to their respective Writings" (Constitution of
      the United States, /us/const/article-I/8/8)


In [52]:
print(oracle.holdings[1] | feist.holdings[2])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was an original work
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java API> was independently created by the author,
      as opposed to copied from other works
      the Fact that <the Java API> possessed at least some minimal degree of
      creativity
      the Fact that <the Java API> was an idea
    GIVEN the ENACTMENTS:
      "the exclusive Right to their respective Writings" (Constitution of
      the United States, /us/const/article-I/8/8)
      "To promote the Progress of Science and useful Arts, by securing for
      limited Times to Authors" (Constitution of the United States,
      /us/const/article-I/8/8)
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which th

It's not obvious that a litigant could really establish all the "GIVEN" Factors listed above in a single case in a court where `oracle.holdings[1]` and `feist.holdings[2]` were both valid law, but if they could, then it seems correct for AuthoritySpoke to conclude that the court would have to find both `the Fact that <the Java API> was an original work` and `the Fact it is false that <the Java API> was copyrightable`.

The union operator is useful for searching for contradictions in a collection of `Holding`s. When two `Holding`s are combined together with the union operator, their union might contradict other `Holding`s that neither of the two original `Holding`s would have contradicted on their own.

## 12. Nuances of Meaning in Holdings

Let's look at one more sentence from the _Oracle_ `Opinion`, so I can point out a few more design decisions AuthoritySpoke makes in representing procedural `Holding`s.

> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

(The "merger" doctrine says that a work is deemed to be "merged" with an uncopyrightable idea if it's essentially the only way to express the idea. "Scenes a faire" is a concept applied mostly to works of fiction, and it means that conventional genre tropes are not copyrightable.)

The quoted sentence is fairly ordinary, as court opinions go, but I found several interesting challenges in creating structered data about its procedural meaning.

1. The sentence describes what the law is "In the Ninth Circuit". You might remember that the court that issued the _Oracle_ opinion was the Federal Circuit, not the Ninth Circuit. So the Federal Circuit is deciding what it thinks that the Ninth Circuit thinks that Congress meant by enacting the statute. The middle layer of this interpretation, in which the Federal Circuit attributes a belief to the Ninth Circuit, is simply absent from the AuthoritySpoke model of the `Holding`. However, future updates to AuthoritySpoke might make it possible to capture this information.

2. The sentence uses the concept of an "affirmative defense", which generally means a defense that the defendant has the burden of proof to establish. I chose to model this concept by writing that if one of the facts that would establish the affirmative defense is present, then it could be established that the copyright was not infringed, but if they are both absent, then the copyright could have been infringed. I'm sure some legal experts would find this too simplistic, and would argue that it's not possible to formalize the concept of an affirmative defense without explicitly mentioning procedural concepts like a burden of proof.

3. The sentence seems to have something to say about what happens if either of two Factors are present, or if both of them are absent. That makes three different Rules. It's not ideal for one sentence to explode into multiple different Python objects when it's formalized, and it's worth wondering whether there would have been a way to pack all the information into a single object. 

4. I noticed that the concept of a copyrighted work being "merged" or being a "scene a faire" are both characteristics intrinsic in the copyrighted work, and don't depend on the characteristics of the allegedly infringing work. So if a work that's "merged" or is a "scene a faire" can't be infringed, but those concepts aren't relevant to copyrightability, then that means there are some works that are "copyrightable" but that can never be infringed by any other work. I suspect that the court's interpretation of these legal categories could confuse future courts and parties, with the result that the "merger" or "scene a faire" concepts could fall through the cracks and be ignored. Would there be a useful way to have AuthoritySpoke flag such anomalies?

The three Holding objects used to represent the sentence from the _Oracle_ opinion can be found in the Appendix below. They're `oracle.holdings[11]` through `oracle.holdings[13]`.

## Appendix: Holding Objects in *Oracle v. Google* and *Lotus v. Borland*

This Appendix will list all of the Holding objects in `oracle.holdings` and `lotus.holdings`. Each `Holding` will be preceded by a passage from the `Opinion` that indicates the `Opinion` has endorsed the `Holding`. In future versions, AuthoritySpoke will give users the ability to explore the text passages in `Opinion`s that provide support for each `Holding`, but that's currently not fully implemented.

To find the full text of the opinions, look in the example_data/opinions/ folder. The text delivered by the CAP API was collected from print sources, so it will contain some Optical Character Recognition errors.

### *Lotus v. Borland* 49 F.3d 807 (1995)

> To establish copyright infringement, a plaintiff must prove "(1) ownership of a valid copyright, and (2) copying of constituent elements of the work that are original."

In [53]:
print(lotus.holdings[0])

the Holding to ACCEPT that the EXCLUSIVE way to reach the fact that
<Borland International> infringed the copyright in <the Lotus menu
command hierarchy> is
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Borland International> infringed the copyright in <the
      Lotus menu command hierarchy>
    GIVEN:
      the Fact that <the Lotus menu command hierarchy> was copyrightable
      the Fact that <Borland International> copied constituent elements of
      <the Lotus menu command hierarchy> that were original
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> To, show ownership of a valid copyright and therefore satisfy Feist’s first prong, a plaintiff must prove that the work as a whole is original and that the plaintiff complied with applicable statutory formalities.

In [54]:
print(lotus.holdings[1])

the Holding to ACCEPT
  the Rule that the court MAY ALWAYS impose the
    RESULT:
      absence of the Fact that <the Lotus menu command hierarchy> was
      copyrightable
    GIVEN:
      absence of the Fact that <the Lotus menu command hierarchy> was an
      original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> In judicial proceedings, a certificate of copyright registration constitutes prima facie evidence of copyrightability and shifts the burden to the defendant to demonstrate why the copyright is not valid.

In [55]:
print(lotus.holdings[2])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Lotus menu command hierarchy> was copyrightable
    GIVEN:
      the Evidence 
        OF:
          the Exhibit in the FORM of certificate of copyright registration
        INDICATING:
          the Fact that <Lotus Development Corporation> registered a copyright
          covering <the Lotus menu command hierarchy>
      absence of the Fact it is false that <the Lotus menu command
      hierarchy> was copyrightable
    GIVEN the ENACTMENT:
      "In any judicial proceedings the certificate of a registration made
      before or within five years after first publication of the work shall
      constitute prima facie evidence of the validity of the copyright and
      of the facts stated in the certificate. The evidentiary weight to be
      accorded the certificate of a registration made thereafter shall be
      within the discretion of the court." (Title 17, /us/usc/t17/s410

> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter; to do this, he or she may either present direct evidence of factual copying or...

In [56]:
print(lotus.holdings[3])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Borland International> copied <the Lotus menu command
      hierarchy> in creating <Quattro's Lotus Emulation Interface>
    GIVEN:
      the Evidence 
        INDICATING:
          the Fact that <Borland International> copied <the Lotus menu command
          hierarchy> in creating <Quattro's Lotus Emulation Interface>
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter; to do this, he or she may either present direct evidence of factual copying or, if that is unavailable, evidence that the alleged infringer had access to the copyrighted work and that the offending and copyrighted works are so similar that the court may infer that there was factual copying (i.e., probative similarity).

In [57]:
print(lotus.holdings[4])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Borland International> copied <the Lotus menu command
      hierarchy> in creating <Quattro's Lotus Emulation Interface>
    GIVEN:
      the Evidence 
        INDICATING:
          the Fact that <Borland International> had access to <the Lotus menu
          command hierarchy>
      the Fact that <Borland International> published <Quattro's Lotus
      Emulation Interface>
      the Evidence 
        INDICATING:
          the Fact that <Quattro's Lotus Emulation Interface> was very similar
          to <the Lotus menu command hierarchy>
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or devic

> To show actionable copying and therefore satisfy Feist’s second prong, a plaintiff must first prove that the alleged infringer copied plaintiffs copyrighted work as a factual matter...The plaintiff must then prove that the copying of copyrighted material was so extensive that it rendered the offending and copyrighted works substantially similar.

In [58]:
print(lotus.holdings[5])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <Borland International> copied constituent elements of
      <the Lotus menu command hierarchy> that were original
    GIVEN:
      the Fact that <Borland International> copied <the Lotus menu command
      hierarchy> in creating <Quattro's Lotus Emulation Interface>
      the Fact that the copying of <Quattro's Lotus Emulation Interface> in
      <the Lotus menu command hierarchy> was so extensive that it rendered
      them substantially similar
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> Section 102(b) states: “In no case does copyright protection for an original work of authorship extend to any idea, procedure, process, system, method of operation, concept, principle, or discovery, regardless of the form in which it is described, explained, illustrated, or embodied in such work.” Because we conclude that the Lotus menu command hierarchy is a method of operation, we do not consider whether it could also be a system, process, or procedure...while original expression is necessary for copyright protection, we do not think that it is alone sufficient. Courts must still inquire whether original expression falls within one of the categories foreclosed from copyright protection by § 102(b), such as being a “method of operation.”

In [59]:
print(lotus.holdings[6])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact it is false that <the Lotus menu command hierarchy> was
      copyrightable
    GIVEN:
      the Fact that <the Lotus menu command hierarchy> was a method of
      operation
    DESPITE:
      the Fact that a text described <the Lotus menu command hierarchy>
      the Fact that <the Lotus menu command hierarchy> was an original work
    GIVEN the ENACTMENTS:
      "In no case does copyright protection for an original work of
      authorship extend to any" (Title 17, /us/usc/t17/s102/b)
      "method of operation" (Title 17, /us/usc/t17/s102/b)


> We hold that the Lotus menu command hierarchy is an uneopyrightable “method of operation.” The Lotus menu command hierarchy provides the means by which users control and operate Lotus 1-2-3. If users wish to copy material, for example, they use the “Copy” command. If users wish to print material, they use the “Print” command. Users must use the command terms to tell the computer what to do. Without the menu command hierarchy, users would not be able to access and control, or indeed make use of, Lotus 1-2-3’s functional capabilities.

In [60]:
print(lotus.holdings[7])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Lotus menu command hierarchy> was a method of
      operation
    GIVEN:
      the Fact that <Lotus 1-2-3> was a computer program
      the Fact that <the Lotus menu command hierarchy> provided the means by
      which users controlled and operated <Lotus 1-2-3>
      the Fact that without <the Lotus menu command hierarchy>, users would
      not have been able to access and control, or indeed make use of,
      <Lotus 1-2-3>’s functional capabilities
      the Fact that for another computer program to by operated in
      substantially the same way as <Lotus 1-2-3>, the other program would
      have to copy <the Lotus menu command hierarchy>
    DESPITE:
      the Fact that the developers of <Lotus 1-2-3> made some expressive
      choices in choosing and arranging the terms in <the Lotus menu command
      hierarchy>
    GIVEN the ENACTMENTS:
      "In no case does copyright

> We do not think that “methods of operation” are limited to abstractions; rather, they are the means by which a user operates something.

In [61]:
print(lotus.holdings[8])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Lotus menu command hierarchy> was a method of
      operation
    GIVEN:
      the Fact that <the Lotus menu command hierarchy> was the means by
      which a person operated <Lotus 1-2-3>
    DESPITE:
      the Fact it is false that <the Lotus menu command hierarchy> was an
      abstraction
    GIVEN the ENACTMENTS:
      "In no case does copyright protection for an original work of
      authorship extend to any" (Title 17, /us/usc/t17/s102/b)
      "method of operation" (Title 17, /us/usc/t17/s102/b)


> In other words, to offer the same capabilities as Lotus 1-2-3, Borland did not have to copy Lotus’s underlying code (and indeed it did not); to 'allow users to operate its programs in substantially the same way, however, Bor-land had to copy the Lotus menu command hierarchy. Thus the Lotus 1-2-3 code is not a uncopyrightable “method of operation.”

In [62]:
print(lotus.holdings[9])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact it is false that <Lotus 1-2-3> was a method of operation
    GIVEN:
      the Fact that <Lotus 1-2-3> was a computer program
      the Fact it is false that the precise formulation of <Lotus 1-2-3>'s
      code was necessary for it to work
    DESPITE:
      the Fact that computer code was necessary for <Lotus 1-2-3> to work
    GIVEN the ENACTMENTS:
      "In no case does copyright protection for an original work of
      authorship extend to any" (Title 17, /us/usc/t17/s102/b)
      "method of operation" (Title 17, /us/usc/t17/s102/b)


### *Oracle v. Google* 750 F.3d 1339 (2014)

> By statute, a work must be “original” to qualify for copyright protection. 17 U.S.C. § 102(a).

In [63]:
print(oracle.holdings[0])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact it is false that <the Java API> was an original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> Original, as the term is used in copyright, means only that the work was independently created by the author (as opposed to copied from other works), and that it possesses at least some minimal degree of creativity.

In [64]:
print(oracle.holdings[1])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was an original work
    GIVEN:
      the Fact that <the Java API> was independently created by the author,
      as opposed to copied from other works
      the Fact that <the Java API> possessed at least some minimal degree of
      creativity
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)


> Copyright protection extends only to the expression of an idea — not to the underlying idea itself...In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

In [65]:
print(oracle.holdings[2])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java API> was an original work
      the Fact that <the Java API> was the expression of an idea
      the Fact it is false that <the Java API> was an idea
    DESPITE:
      the Fact that <the Java API> was essentially the only way to express
      the idea that it embodied
      the Fact that <the Java API> was a scene a faire
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorshi

> The literal elements of a computer program are the source code and object code.

In [66]:
print(oracle.holdings[3])
print("\n")
print(oracle.holdings[4])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was a literal element of <the Java
      language>
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was the source code of <the Java
      language>
    GIVEN the ENACTMENTS:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in

> It is well established that copyright protection can extend to both literal and non-literal elements of a computer program. See Altai 982 F.2d at 702.

In [67]:
print(oracle.holdings[5])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was a literal element of <the Java
      language>
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in s

> The non-literal components of a computer program include, among other things, the program’s sequence, structure, and organization, as well as the program’s user interface.

In [68]:
print(oracle.holdings[6])
print("\n")
print(oracle.holdings[7])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was a non-literal element of <the Java
      language>
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was the sequence, structure, and
      organization of <the Java language>
    GIVEN the ENACTMENTS:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explaine

> It is well established that copyright protection can extend to both literal and non-literal elements of a computer program...As discussed below, whether the non-literal elements of a program “are protected depends on whether, on the particular facts of each case, the component in question qualifies as an expression of an idea, or an idea itself.”

In [69]:
print(oracle.holdings[8])
print("\n")
print(oracle.holdings[9])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was a non-literal element of <the Java
      language>
      the Fact that <the Java API> was the expression of an idea
      the Fact it is false that <the Java API> was an idea
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, c

> On appeal, Oracle argues that the district court’s reliance on Lotus is misplaced because it is distinguishable on its facts and is inconsistent with Ninth Circuit law. We agree. First, while the defendant in Lotus did not copy any of the underlying code, Google concedes that it copied portions of Oracle’s declaring source code verbatim. Second, the Lotus court found that the commands at issue there (copy, print, etc.) were not creative, but it is undisputed here that the declaring code and the structure and organization of the API packages are both creative and original. Finally, while the court in Lotus found the commands at issue were “essential to operating” the system, it is undisputed that— other than perhaps as to the three core packages — Google did not need to copy the structure, sequence, and organization of the Java API packages to write programs in the Java language.\nMore importantly, however, the Ninth Circuit has not adopted the court’s “method of operation” reasoning in Lotus, and we conclude that it is inconsistent with binding precedent.

In [70]:
print(oracle.holdings[10])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was a set of application programming
      interface declarations
      the Fact that <the Java API> was an original work
      the Fact that <the Java API> was a non-literal element of <the Java
      language>
      the Fact that <the Java API> was the expression of an idea
      the Fact it is false that <the Java API> was essentially the only way
      to express the idea that it embodied
      the Fact that <the Java API> was creative
      the Fact that it was possible to use <the Java language> without
      copying <the Java API>
    DESPITE:
      the Fact that <the Java API> was a method of operation
      the Fact that <the Java API> contained short phrases
      the Fact that <the Java API> became so popular that it was the
      

> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement.

In [71]:
print(oracle.holdings[11])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <Google> infringed the copyright on <the
      Java API>
    GIVEN:
      the Fact that <the Java API> was a scene a faire
    DESPITE:
      the Fact that <the Java API> was copyrightable
    GIVEN the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in such
      work." (Title 17, /us/usc/t17/s102/b)
    DESPITE the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid 

> In the Ninth Circuit, while questions regarding originality are considered questions of copyrightability, concepts of merger and scenes a faire are affirmative defenses to claims of infringement...Under the merger doctrine, a court will not protect a copyrighted work from infringement if the idea contained therein can be expressed in only one way.

In [72]:
print(oracle.holdings[12])
print("\n")
print(oracle.holdings[13])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <Google> infringed the copyright on <the
      Java API>
    GIVEN:
      the Fact that <the Java API> was essentially the only way to express
      the idea that it embodied
    DESPITE:
      the Fact that <the Java API> was copyrightable
    GIVEN the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in such
      work." (Title 17, /us/usc/t17/s102/b)
    DESPITE the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise 

#### A Missing Holding

The following text represents a rule posited by the Oracle court, but it's not currently possible to create a corresponding Holding object, because AuthoritySpoke doesn't yet include "Argument" objects.

> Google responds that Oracle waived its right to assert copyrightability based on the 7,000 lines of declaring code by failing “to object to instructions and a verdict form that effectively eliminated that theory from the case.” Appellee Br. 67...We find that Oracle did not waive arguments based on Google’s literal copying of the declaring code.

> Regardless of when the analysis occurs, we conclude that merger does not apply on the record before us...We have recognized, however, applying Ninth Circuit law, that the “unique arrangement of computer program expression ... does not merge with the process so long as alternate expressions are available.”...The evidence showed that Oracle had “unlimited options as to the selection and arrangement of the 7000 lines Google copied.”...This was not a situation where Oracle was selecting among preordained names and phrases to create its packages.

In [73]:
print(oracle.holdings[14])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was essentially the only way
      to express the idea that it embodied
    GIVEN:
      the Fact that <Sun Microsystems> created <the Java API>
      the Fact that when creating <the Java API>, <Sun Microsystems> could
      have selected and arranged its names and phrases in unlimited
      different ways
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
   

> the relevant question for copyright-ability purposes is not whether the work at issue contains short phrases — as literary works often do — but, rather, whether those phrases are creative.

In [74]:
print(oracle.holdings[15])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java API> was a literary work
      the Fact that the short phrases in <the Java API> was creative
    DESPITE:
      the Fact that <the Java API> contained short phrases
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENTS:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is des

> In the computer context, “the scene a faire doctrine denies protection to program elements that are dictated by external factors such as ‘the mechanical specifications of the computer on which a particular program is intended to run’ or ‘widely accepted programming practices within the computer industry. Like merger, the focus of the scenes a faire doctrine is on the circumstances presented to the creator, not the copier.

In [75]:
print(oracle.holdings[16])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was a scene a faire
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was an element of <the Java language>
      the Fact that the creation of <the Java API> was dictated by external
      factors such as the mechanical specifications of the computer on which
      <the Java language> was intended to run or widely accepted programming
      practices within the computer industry
    GIVEN the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in such
      work." (Title 17, /us/usc/t17/s102/b)
    DESPITE the ENACTMENT:
      "Copyright protection subsists, in accordan

> Specifically, we find that Lotus is inconsistent with Ninth Circuit case law recognizing that the structure, sequence, and organization of a computer program is eligible for copyright protection where it qualifies as an expression of an idea, rather than the idea itself.

In [76]:
print(oracle.holdings[17])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java language> was a computer program
      the Fact that <the Java API> was the structure, sequence, and
      organization of <the Java language>
      the Fact that <the Java API> was the expression of an idea
      the Fact it is false that <the Java API> was an idea
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of

> an original work — even one that serves a function — is entitled to copyright protection as long as the author had multiple ways to express the underlying idea. Section 102(b) does not, as Google seems to suggest, automatically deny copyright protection to elements of a computer program that are functional.

In [77]:
print(oracle.holdings[18])

the Holding to ACCEPT
  the Rule that the court MUST ALWAYS impose the
    RESULT:
      the Fact that <the Java API> was copyrightable
    GIVEN:
      the Fact that <the Java API> was an original work
      the Fact that <Sun Microsystems> was the author of <the Java API>
      the Fact that when creating <the Java API>, <Sun Microsystems> had
      multiple ways to express its underlying idea
    DESPITE:
      the Fact that <the Java API> served a function
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process

> Until either the Supreme Court or Congress tells us otherwise, we are bound to respect the Ninth Circuit’s decision to afford software programs protection under the copyright laws. We thus decline any invitation to declare that protection of software programs should be the domain of patent law, and only patent law.

In [78]:
print(oracle.holdings[19])

the Holding to ACCEPT
  the Rule that the court MAY SOMETIMES impose the
    RESULT:
      the Fact that <the Java language> was copyrightable
    GIVEN:
      the Fact that <the Java language> was a computer program
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in
      original works of authorship fixed in any tangible medium of
      expression, now known or later developed, from which they can be
      perceived, reproduced, or otherwise communicated, either directly or
      with the aid of a machine or device." (Title 17, /us/usc/t17/s102/a)
    DESPITE the ENACTMENT:
      "In no case does copyright protection for an original work of
      authorship extend to any idea, procedure, process, system, method of
      operation, concept, principle, or discovery, regardless of the form in
      which it is described, explained, illustrated, or embodied in such
      work." (Title 17, /us/usc/t17/s102/b)
